In [4]:
import sys
import math 
import time 
from tqdm import tqdm
B = 4
    

In [3]:
class Node(object): #node class
    def __init__(self):
        self.id = 0
        # for internal nodes
        self.child_nodes = []
        # for leaf nodes
        self.data_points = []
        self.parent = None
        self.MBR = {
            'x1': -1,
            'y1': -1,
            'x2': -1,
            'y2': -1,
        }
    def perimeter(self):
        # only calculate the half perimeter here
        return (self.MBR['x2'] - self.MBR['x1']) + (self.MBR['y2'] - self.MBR['y1'])

    def is_overflow(self):
        if self.is_leaf():
            if self.data_points.__len__() > B: #Checking overflows of data points, B is the upper bound.
                return True
            else:
                return False
        else:
            if self.child_nodes.__len__() > B: #Checking overflows of child nodes, B is the upper bound.
                return True
            else:
                return False

    def is_root(self):
        if self.parent is None:
            return True
        else:
            return False

    def is_leaf(self):
        if self.child_nodes.__len__() == 0:
            return True
        else:
            return False



In [5]:
class RTree(object): #R tree class
    def __init__(self):
        self.root = Node() #Create a root
    def insert(self, u, p): # insert p(data point) to u (MBR)
        if u.is_leaf(): 
            self.add_data_point(u, p) #add the data point and update the corresponding MBR
            if u.is_overflow():
                self.handle_overflow(u) #handel overflow for leaf nodes
        else:
            v = self.choose_subtree(u, p) #choose a subtree to insert the data point to miminize the perimeter sum
            self.insert(v, p) #keep continue to check the next layer recursively
            self.update_mbr(v) #update the MBR for inserting the data point

    def choose_subtree(self, u, p): 
        if u.is_leaf(): #find the leaf and insert the data point
            return u
        else:
            min_increase = sys.maxsize #set an initial value
            best_child = None
            for child in u.child_nodes: #check each child to find the best node to insert the point 
                if min_increase > self.peri_increase(child, p):
                    min_increase = self.peri_increase(child, p)
                    best_child = child
            return best_child

    def peri_increase(self, node, p): # calculate the increase of the perimeter after inserting the new data point
        # new perimeter - original perimeter = increase of perimeter
        origin_mbr = node.MBR
        x1, x2, y1, y2 = origin_mbr['x1'], origin_mbr['x2'], origin_mbr['y1'], origin_mbr['y2']
        increase = (max([x1, x2, p['x']]) - min([x1, x2, p['x']]) +
                    max([y1, y2, p['y']]) - min([y1, y2, p['y']])) - node.perimeter()
        return increase


    def handle_overflow(self, u):
        u1, u2 = self.split(u) #u1 u2 are the two splits returned by the function "split"
        # if u is root, create a new root with s1 and s2 as its' children
        if u.is_root():
            new_root = Node()
            self.add_child(new_root, u1)
            self.add_child(new_root, u2)
            self.root = new_root
            self.update_mbr(new_root)
        # if u is not root, delete u, and set s1 and s2 as u's parent's new children
        else:
            w = u.parent
            # copy the information of s1 into u
            w.child_nodes.remove(u)
            self.add_child(w, u1) #link the two splits and update the corresponding MBR
            self.add_child(w, u2)
            if w.is_overflow(): #check the parent node recursively
                self.handle_overflow(w)
            
    def split(self, u):
        # split u into s1 and s2
        best_s1 = Node()
        best_s2 = Node()
        best_perimeter = sys.maxsize
        # u is a leaf node
        if u.is_leaf():
            m = u.data_points.__len__()
            # create two different kinds of divides
            divides = [sorted(u.data_points, key=lambda data_point: data_point['x']),
                       sorted(u.data_points, key=lambda data_point: data_point['y'])] #sorting the points based on X dimension and Y dimension
            for divide in divides:
                for i in range(math.ceil(0.4 * B), m - math.ceil(0.4 * B) + 1): #check the combinations to find a near-optimal one
                    s1 = Node()
                    s1.data_points = divide[0: i]
                    self.update_mbr(s1)
                    s2 = Node()
                    s2.data_points = divide[i: divide.__len__()]
                    self.update_mbr(s2)
                    if best_perimeter > s1.perimeter() + s2.perimeter(): 
                        best_perimeter = s1.perimeter() + s2.perimeter()
                        best_s1 = s1
                        best_s2 = s2

        # u is a internal node
        else:
            # create four different kinds of divides
            m = u.child_nodes.__len__()
            divides = [sorted(u.child_nodes, key=lambda child_node: child_node.MBR['x1']), #sorting based on MBRs
                       sorted(u.child_nodes, key=lambda child_node: child_node.MBR['x2']),
                       sorted(u.child_nodes, key=lambda child_node: child_node.MBR['y1']),
                       sorted(u.child_nodes, key=lambda child_node: child_node.MBR['y2'])]
            for divide in divides:
                for i in range(math.ceil(0.4 * B), m - math.ceil(0.4 * B) + 1): #check the combinations
                    s1 = Node()
                    s1.child_nodes = divide[0: i]
                    self.update_mbr(s1)
                    s2 = Node()
                    s2.child_nodes = divide[i: divide.__len__()]
                    self.update_mbr(s2)
                    if best_perimeter > s1.perimeter() + s2.perimeter():
                        best_perimeter = s1.perimeter() + s2.perimeter()
                        best_s1 = s1
                        best_s2 = s2

        for child in best_s1.child_nodes:
            child.parent = best_s1
        for child in best_s2.child_nodes:
            child.parent = best_s2

        return best_s1, best_s2


    def add_child(self, node, child):
        node.child_nodes.append(child) #add child nodes to the current parent (node) and update the MBRs. It is used in handeling overflows
        child.parent = node
        if child.MBR['x1'] < node.MBR['x1']:
            node.MBR['x1'] = child.MBR['x1']
        if child.MBR['x2'] > node.MBR['x2']:
            node.MBR['x2'] = child.MBR['x2']
        if child.MBR['y1'] < node.MBR['y1']:
            node.MBR['y1'] = child.MBR['y1']
        if child.MBR['y2'] > node.MBR['y2']:
            node.MBR['y2'] = child.MBR['y2']
    # return the child whose MBR requires the minimum increase in perimeter to cover p

    def add_data_point(self, node, data_point): #add data points and update the the MBRS
        node.data_points.append(data_point)
        if data_point['x'] < node.MBR['x1']:
            node.MBR['x1'] = data_point['x']
        if data_point['x'] > node.MBR['x2']:
            node.MBR['x2'] = data_point['x']
        if data_point['y'] < node.MBR['y1']:
            node.MBR['y1'] = data_point['y']
        if data_point['y'] > node.MBR['y2']:
            node.MBR['y2'] = data_point['y']


    def update_mbr(self, node): #update MBRs when forming a new MBR. It is used in checking the combinations and update the root
        x_list = []
        y_list = []
        if node.is_leaf():
            x_list = [point['x'] for point in node.data_points]
            y_list = [point['y'] for point in node.data_points]
        else:
            x_list = [child.MBR['x1'] for child in node.child_nodes] + [child.MBR['x2'] for child in node.child_nodes]
            y_list = [child.MBR['y1'] for child in node.child_nodes] + [child.MBR['y2'] for child in node.child_nodes]
        new_mbr = {
            'x1': min(x_list),
            'x2': max(x_list),
            'y1': min(y_list),
            'y2': max(y_list)
        }
        node.MBR = new_mbr


In [10]:
def read_dataset(file_path):
    dataset = []
    with open(file_path, 'r') as file:
        for line in file:
            line = line.split(' ')
            point = {
                'id': int(line[0]),
                'x': float(line[1]),
                'y': float(line[2])
            }
            dataset.append(point)
    return dataset

def read_queries(file_path):
    queries = []
    with open(file_path, 'r') as file:
        for line in file:
            line = line.split(' ')
            query = {
                'id': int(line[0]),
                'x_start': float(line[1]),
                'x_end': float(line[2]),
                'y_start': float(line[3]),
                'y_end': float(line[4])
            }
            queries.append(query)
    return queries

def range_query(root, query):
    result = []

    def search(node, query):
        if node.is_leaf():
            for data_point in node.data_points:
                if query['x_start'] <= data_point['x'] <= query['x_end'] and query['y_start'] <= data_point['y'] <= query['y_end']:
                    result.append(data_point)
        else:
            for child in node.child_nodes:
                if overlap(child.MBR, query):
                    search(child, query)

    search(root, query)
    return len(result)

def overlap(mbr, query):
    if query['x_end'] < mbr['x1'] or query['x_start'] > mbr['x2']:
        return False
    if query['y_end'] < mbr['y1'] or query['y_start'] > mbr['y2']:
        return False
    return True

def build_rtree(dataset):
    rtree = RTree()
    total_points = len(dataset)

    with tqdm(total=total_points, desc='Building R-tree') as pbar:
        for i, point in enumerate(dataset):
            rtree.insert(rtree.root, point)
            pbar.update(1)
            time.sleep(0.01)  # Optional: Sleep to control the progress bar speed

    return rtree




In [11]:
def sequential_scan(dataset, query_file):
    query_results = []
    with open(query_file, 'r') as file:
        for line in tqdm(file, desc='Sequential Scan', total=200):
            line = line.split(' ')
            query_id = int(line[0])
            x1, x2 = float(line[1]), float(line[2])
            y1, y2 = float(line[3]), float(line[4])
            
            count = 0
            for point in dataset:
                if x1 <= point['x'] <= x2 and y1 <= point['y'] <= y2:
                    count += 1
            
            query_results.append((query_id, count))
    
    return query_results

    







In [13]:
dataset_file = '/content/sample_data/Dataset.txt'
queries_file = '/content/sample_data/quiries.txt'
output_file = 'output.txt'

dataset = read_dataset(dataset_file)
queries = read_queries(queries_file)
# measure and show Rtree building time
go_build = time.time()
rtree = build_rtree(dataset)
end_build = time.time() - go_build
# Sequential scan benchmark
start_time = time.time()
seq_result = sequential_scan(dataset, queries_file)
sequential_scan_time = time.time() - start_time


Sequential Scan: 100%|██████████| 200/200 [00:04<00:00, 49.08it/s]


In [19]:
# Range queries with R-tree
total_query_time = 0
query_results = []

for query in tqdm(queries, desc='Range Queries'):
  start_time = time.time()
  count = range_query(rtree.root, query)
  query_time = time.time() - start_time
  total_query_time += query_time
  print(count)
  query_results.append(count)
avg_query_time = total_query_time / len(queries)



Range Queries: 100%|██████████| 200/200 [00:00<00:00, 2220.78it/s]

9
12
10
5
6
13
13
7
7
10
3
8
12
6
6
8
6
7
8
6
6
10
6
4
7
9
13
12
8
8
10
11
4
4
4
6
9
6
4
8
8
6
4
10
6
6
8
10
7
4
9
9
8
9
10
4
6
3
4
8
8
13
3
5
7
9
8
8
9
5
8
6
9
9
3
9
10
7
8
14
6
14
3
7
3
3
5
11
3
8
10
2
9
8
5
2
17
9
2
7
10
4
8
7
7
9
12
11
5
7
11
10
4
6
4
11
10
7
10
7
9
5
11
14
8
7
10
7
5
5
5
6
8
8
10
7
5
6
10
6
8
7
7
7
8
9
4
11
2
8
11
4
3
6
10
5
6
10
5
7
9
4
8
9
8
6
7
6
6
9
6
14
9
6
9
2
4
9
6
7
6
3
6
14
10
5
8
12
8
11
5
3
11
2
11
6
8
7
8
6


In [23]:
with open(output_file, 'w') as file:
  file.write(f"Sequential Scan Time: {sequential_scan_time} seconds\n")
  file.write(f"Total RTree building Time: {end_build} seconds\n")
  for s,q in seq_result:
    file.write(f"returned counts with sequential scan: {q}\n")
  total_time = 0
  file.write("Query Results:\n")
  for result in query_results:
    file.write(f"{result}\n")
    total_time = total_query_time
  avg_time = avg_query_time
  file.write(f"Total Query Time: {total_time} seconds\n")
  file.write(f"Average Query Time: {avg_time} seconds\n")
  tim = sequential_scan_time/(math.ceil(total_time))
  file.write(f"R-Tree Based Search is {tim} times faster than sequential scan")